<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/bgRemoval/tests/PaddleSeg_matting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PaddleSeg - matting + BG-inpainting

[GitHub](https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.5/Matting)


# 1. linking nextcloud

Connecting to the external NextCloud drive 

In [ ]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='bgRemoval',library='MODNet')

# 2. Clone the GIT repo and download the model